# 0. Initialisation

In [1]:
// Initialisation chemin des données
val path = "/home/georges/IASDfev20/3_BigData/DarioC/TP/MasterIASD_Flights_Project/data/"
val pathA = "/home/georges/IASDfev20/3_BigData/DarioC/TP_all/data/ONTIME/"
val pathQ = "/home/georges/IASDfev20/3_BigData/DarioC/TP_all/data/QCLCD/"

// Initialisation des variables de date
val date1 = "201311"
val date2 = "201511"
val date3 = "201711"

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.30:4041
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1608204283559)
SparkSession available as 'spark'


path: String = /home/georges/IASDfev20/3_BigData/DarioC/TP/MasterIASD_Flights_Project/data/
pathA: String = /home/georges/IASDfev20/3_BigData/DarioC/TP_all/data/ONTIME/
pathQ: String = /home/georges/IASDfev20/3_BigData/DarioC/TP_all/data/QCLCD/
date1: String = 201311
date2: String = 201511
date3: String = 201711


# 1. Liste des aéroports de référence
Les fichiers de vols sont utilisés pour constituer une base de référence des aéroports, via leur code à 3 lettres.

Cette liste (une par année par exemple) permet de délimiter le périmètre des stations météo nécessaires.

In [2]:
// liste des aéroports (sur la base de l'origine et de la destination des vols)
// utilisation de 3 dates nov 2013, 2015 et 2017

val data_airp1 = spark.read.format("csv").option("header", "true")
                      .load(pathA+date1+"_T_ONTIME_REPORTING.csv")
                      .select("origin","dest")
val data_airp1o = data_airp1.withColumnRenamed("origin","CallSign").select("CallSign").distinct
val data_airp1d = data_airp1.withColumnRenamed("dest","CallSign").select("CallSign").distinct

val list_airp1 = data_airp1o.union(data_airp1d).distinct

val data_airp2 = spark.read.format("csv").option("header", "true")
                      .load(pathA+date2+"_T_ONTIME_REPORTING.csv")
                      .select("origin","dest")
val data_airp2o = data_airp2.withColumnRenamed("origin","CallSign").select("CallSign").distinct
val data_airp2d = data_airp2.withColumnRenamed("dest","CallSign").select("CallSign").distinct
val list_airp2 = data_airp2o.union(data_airp2d).distinct

val data_airp3 = spark.read.format("csv").option("header", "true")
                      .load(pathA+date3+"_T_ONTIME_REPORTING.csv")
                      .select("origin","dest")
val data_airp3o = data_airp3.withColumnRenamed("origin","CallSign").select("CallSign").distinct
val data_airp3d = data_airp3.withColumnRenamed("dest","CallSign").select("CallSign").distinct
val list_airp3 = data_airp3o.union(data_airp3d).distinct

val liste_airp = list_airp1.union(list_airp2).union(list_airp3).distinct
                          .orderBy($"CallSign".asc)
println(liste_airp.count())

320


data_airp1: org.apache.spark.sql.DataFrame = [origin: string, dest: string]
data_airp1o: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
data_airp1d: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
list_airp1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
data_airp2: org.apache.spark.sql.DataFrame = [origin: string, dest: string]
data_airp2o: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
data_airp2d: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
list_airp2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
data_airp3: org.apache.spark.sql.DataFrame = [origin: string, dest: string]
data_airp3o: org.apache.spark.sql.Dat...


# 2. Préparation des données météo : Weather Observation Table (WOT)

La mise en forme des données météo consiste :

* Etape 1 - Dans un premier temps à lire les fichiers récupérer le lien dans les fichiers stations.txt entre le code aéroport de trois letttres et le Wban d'identification des stations. Cela permet :
    * d'une part de faire limiter le périmètre des stations à la liste des aéroports de référence => Weather Observation Table (OT)
    * d'autre part de croiser les données météo avec celle des vols.
    
* Etape 2 - Dans un second temps à construire, pour chaque station et chaque jour, le relevé des informations météo pour chaque heure (de 0 à 23h) :
    * les stations/dates conservées sont celles où l'on retrouve un relevé complet sur 24 heures ;
    * les informations météo utilisées (pression, température...) sont de niveau horaire, journalier ou mensuelles et sont conservées pour chaque heure dans un vecteur et indéxées pour les informations de nature qualitative.

## Etape 1/2 - Lecture des fichiers QCLCD => Weather Observation File (WOF)
Réduction aux stations correspondant aux aéeroports

### (i) Fichier des stations restreint à la liste de référence des aéroports
A noter qu'il existe des aéroports pour lesquels aucune station n'est trouvée

In [3]:
// novembre 2013

val data_station0 = spark.read.format("csv").option("header", "true").option("delimiter", "|")
                              .load(pathQ+date1+"station.txt")
                              .withColumnRenamed("CallSign", "c_CallSign")

val data_station = data_station0.join(liste_airp,data_station0("c_CallSign") === liste_airp("CallSign"),"inner")
                                .select("CallSign", "WBAN","Name","State","GroundHeight", "Barometer", "TimeZone")

print("Nb stations : ") 
println(data_station.count())
data_station.show(5, false)


// Identification des aéroports sans station
val data_missed_airp = liste_airp.join(data_station0, liste_airp("CallSign")===data_station0("c_CallSign"),"left")
                                 .filter("c_CallSign is null")
                                 .select("CallSign")
print("Nb missed airports : ")
println(data_missed_airp.count())
data_missed_airp.show(false)

Nb stations : 312
+--------+-----+-------------------+-----+------------+---------+--------+
|CallSign|WBAN |Name               |State|GroundHeight|Barometer|TimeZone|
+--------+-----+-------------------+-----+------------+---------+--------+
|BGM     |04725|BINGHAMTON         |NY   |1595        |1630     |-5      |
|INL     |14918|INTERNATIONAL FALLS|MN   |1183        |1185     |-6      |
|MSY     |12916|NEW ORLEANS        |LA   |4           |7        |-6      |
|GEG     |24157|SPOKANE            |WA   |2353        |2384     |-8      |
|SNA     |93184|SANTA ANA          |CA   |54          |52       |-8      |
+--------+-----+-------------------+-----+------------+---------+--------+
only showing top 5 rows

Nb missed airports : 8
+--------+
|CallSign|
+--------+
|PSE     |
|PPG     |
|SCE     |
|SPN     |
|UST     |
|BKG     |
|FCA     |
|CLD     |
+--------+



data_station0: org.apache.spark.sql.DataFrame = [WBAN: string, WMO: string ... 13 more fields]
data_station: org.apache.spark.sql.DataFrame = [CallSign: string, WBAN: string ... 5 more fields]
data_missed_airp: org.apache.spark.sql.DataFrame = [CallSign: string]


### (ii) Fichier des relevés mensuels (monthly.txt)

In [4]:
// novembre 2013
val data_monthly = spark.read.format("csv").option("header", "true").option("delimiter", ",")
                             .load(pathQ+date1+"monthly.txt")
                             .withColumnRenamed("WBAN","c_WBAN")
print("monthly data : ")
println(data_monthly.count())

/* Finaliser les données mensuelles à récupérer 

data_monthly.printSchema
data_monthly.select("Wban","YearMonth","AvgMaxTemp","AvgMinTemp","AvgTemp","HeatingDegreeDays",
                    "HDDSeasonToDate","ThunderstormDays","HeavyFogDays").show(false)
*/


monthly data : 1225


data_monthly: org.apache.spark.sql.DataFrame = [c_WBAN: string, YearMonth: string ... 48 more fields]


### (iii) Fichier des relevés quotidiens (daily.txt)

In [5]:
// novembre 2013
val data_daily = spark.read.format("csv").option("header", "true").option("delimiter", ",")
                             .load(pathQ+date1+"daily.txt")
                             .withColumnRenamed("WBAN","c_WBAN")
                             .select("c_WBAN","YearMonthDay","Tmax","Tmin","Tavg","DewPoint","WetBulb", "Heat",
                                     "Cool","Sunrise","Sunset","CodeSum","Depth","Water1", "SnowFall",
                                     "StnPressure", "SeaLevel", "ResultSpeed", "ResultDir", "AvgSpeed", "Max5Speed",
                                     "Max5Dir", "Max2Speed", "Max2Dir")
print("daily data : ")
println(data_daily.count())

daily data : 36750


data_daily: org.apache.spark.sql.DataFrame = [c_WBAN: string, YearMonthDay: string ... 22 more fields]


### (iv) Fichier des relevés horaires (hourly.txt)

In [6]:
// novembre 2013
val data_hourly = spark.read.format("csv").option("header", "true").option("delimiter", ",")
                       .load(pathQ+date1+"hourly.txt")
                       .withColumnRenamed("WBAN","c_WBAN")
                       .withColumn("c_Month",substring($"Date", 1, 6))
                       .select("c_WBAN","Date","c_Month","Time","StationType","SkyCondition","Visibility","WeatherType",
                               "DryBulbFarenheit","WetBulbFarenheit","DewPointFarenheit",
                               "WindSpeed","WindDirection","StationPressure","SeaLevelPressure","HourlyPrecip")
print("hourly data : ")
println(data_hourly.count())
/*
"ValueForWindCharacter" à voir ?
"PressureTendency","PressureChange" très peu renseignées (10 obs)
data_hourly.select(
"PressureTendency","PressureChange"
).where($"PressureTendency" !== " ").show(50,false)
"RecordType" : signification ?
*/                        

hourly data : 4060568


data_hourly: org.apache.spark.sql.DataFrame = [c_WBAN: string, Date: string ... 14 more fields]


### (v) Fichier des précipitations (precip.txt)

In [ ]:
/* L'information semble être déjà reprise dans la variable HourlyPrecip du fichier hourly.txt

val data_precip = spark.read.format("csv").option("header", "true").option("delimiter", ",")
                             .load(pathQ+date1+"precip.txt")

data_hourly.select("Wban", "Date", "Time", "HourlyPrecip").where($"HourlyPrecip" !== " ")
.join(data_precip.select("Wban","YearMonthDay","Hour","Precipitation"), 
     data_hourly("Wban")===data_precip("Wban") && data_hourly("Date")===data_precip("YearMonthDay"), "inner")
.where($"Precipitation" !== " ")
.show(80,false)
*/

### (vi) => Dataframe Weather Observation File (WOF)

In [7]:
// DATAFRAME DE SYNTHESE regroupant l'ensemble des informations météo pertinentes
val data_meteo_a = data_station.join(data_hourly, data_station("WBAN") === data_hourly("c_WBAN"), "inner")
                               .drop("c_WBAN")
print("meteo_a : ")
println(data_meteo_a.count())

val data_meteo_b = data_meteo_a.join(data_daily, data_meteo_a("WBAN") === data_daily("c_WBAN") &&
                                                 data_meteo_a("Date") === data_daily("YearMonthDay"), "left")
                               .drop("c_WBAN")
print("meteo_b : ")
println(data_meteo_b.count())

val WOF_201311= data_meteo_b.join(data_monthly, data_meteo_b("WBAN") === data_monthly("c_WBAN") &&
                                                data_meteo_b("c_Month") === data_monthly("yearMonth"), "left")
                            .drop("c_WBAN","c_Month", "YearMonth","YearMonthDay")
print("WOF_201311 : ")
println(WOF_201311.count())

// WOT_201311.printSchema       

meteo_a : 296924
meteo_b : 296924
WOF_201311 : 296924


data_meteo_a: org.apache.spark.sql.DataFrame = [CallSign: string, WBAN: string ... 20 more fields]
data_meteo_b: org.apache.spark.sql.DataFrame = [CallSign: string, WBAN: string ... 43 more fields]
WOF_201311: org.apache.spark.sql.DataFrame = [CallSign: string, WBAN: string ... 89 more fields]


In [8]:
/* Remarque : présence de couples WBAN/Date dans le fichier hourly mais absents de daily ou de monthly.

A voir si l'on se restreint aux observations pour lesquelles nous disposons à la fois des données horaires,
quotidiennes et mensuelles. Ici le périmètre est celui - plus large des données horaires avec valeurs
manquantes sur les daily ou monthly donc */

// écarts hourly et daily
val data_h = data_meteo_a.select("WBAN","Date")
val data_d = data_daily.withColumnRenamed("c_WBAN", "WBAN")
                               .withColumnRenamed("YearMonthDay","Date")
                               .select("WBAN","Date")

val data_sub = data_h.except(data_d)
print("Nb Wban absents data daily : ")
println(data_sub.count())

// écarts hourly et monthly
val data_h2 = data_meteo_a.select("WBAN","c_Month")
val data_m = data_monthly.withColumnRenamed("c_WBAN", "WBAN")
                               .withColumnRenamed("YearMonth","c_Month")
                               .select("WBAN","c_Month")

val data_sub2 = data_h2.except(data_m)
print("Nb Wban absents data monthly :")
println(data_sub2.count())


Nb Wban absents data daily : 784
Nb Wban absents data monthly :27


data_h: org.apache.spark.sql.DataFrame = [WBAN: string, Date: string]
data_d: org.apache.spark.sql.DataFrame = [WBAN: string, Date: string]
data_sub: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [WBAN: string, Date: string]
data_h2: org.apache.spark.sql.DataFrame = [WBAN: string, c_Month: string]
data_m: org.apache.spark.sql.DataFrame = [WBAN: string, c_Month: string]
data_sub2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [WBAN: string, c_Month: string]


## Etape 2/2 - Mise en forme des données météo => Weather Observation Table (WOT)
Cette étape se déroule en deux temps :
* Ajustement du périmètre des relevés : uniquement les stations et les jours pour lesquels les relevés horaires sont disponibles chaque heure sur 24 heures (0 à 23h).
* Création d'une colonne Weather_h (Array) reprenant les 24 relevés horaires

In [8]:
// Echantillon de travail : on se limite ici à quelques infos météo 

val echan_hourly = WOF_201311.select("CallSign", "State", "Date", "Time",
                                     "SkyCondition", "WindSpeed","StationPressure","Tmax","Tmin","AvgTemp")
echan_hourly.show(10, false)
println(echan_hourly.count())

+--------+-----+--------+----+--------------------+---------+---------------+----+----+-------+
|CallSign|State|Date    |Time|SkyCondition        |WindSpeed|StationPressure|Tmax|Tmin|AvgTemp|
+--------+-----+--------+----+--------------------+---------+---------------+----+----+-------+
|MEI     |MS   |20131101|0056|SCT010 BKN017 OVC060| 7       |29.52          |74  |48  |51.2   |
|MEI     |MS   |20131101|0058|FEW010 BKN014 OVC060| 8       |29.52          |74  |48  |51.2   |
|MEI     |MS   |20131101|0113|FEW010 BKN015 BKN020| 8       |29.52          |74  |48  |51.2   |
|MEI     |MS   |20131101|0123|FEW012 SCT020 BKN055| 5       |29.51          |74  |48  |51.2   |
|MEI     |MS   |20131101|0158|CLR                 | 5       |29.51          |74  |48  |51.2   |
|MEI     |MS   |20131101|0258|FEW010 SCT050       | 5       |29.51          |74  |48  |51.2   |
|MEI     |MS   |20131101|0316|FEW008 SCT050       | 0       |29.51          |74  |48  |51.2   |
|MEI     |MS   |20131101|0330|BKN008 BKN

echan_hourly: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 8 more fields]


### (i) Limitation du périmètre aux stations/Date avec 24 relevés horaires

En vue du croisement avec les données sur les vols :
* Création de la variable DateH  correspondant à Date en format date

* Création de la variable TimeH correspondant à l'heure pleine la plus proche de l'heure de relevé (Time)

* Limitation aux dates pour lesquelles il existe 24 relevés météo par jour (un par heure). En effet, dans certains cas, il y a des relevés intermédiaires ou il y en a moins que 24. La variable _t_ correspond au numéro de l'heure.

In [57]:
import org.apache.spark.sql.functions.unix_timestamp
import org.apache.spark.sql.functions.{to_date, to_timestamp}
import org.apache.spark.sql.functions.{hour,minute,second}

import org.apache.spark.sql.expressions.Window

// Fonctions d'agrégation (window)
val w_deltaNb = Window
                    .partitionBy("CallSign","Date","Delta")
                    .orderBy($"Delta" .asc)

val w_deltaRg = Window
                    .partitionBy("CallSign","Date")
                    .orderBy($"Delta" .asc)

// Gestion des dates et limitation à 24 du nombre de relevés (CountData)
val hourly = echan_hourly.withColumn("DateH",to_date(unix_timestamp($"Date","yyyyMMdd").cast("timestamp")))
                         .withColumn("DTime",from_unixtime(unix_timestamp($"Time","HHmm"),"HH:mm:ss"))
                         .withColumn("minute", minute($"DTime"))
                         .withColumn("new_min", (floor($"minute"/60))*60)
                         .withColumn("min_add", $"new_min"- $"minute")
                         .withColumn("TimeH", from_unixtime(unix_timestamp($"DTime","HH:mm:ss")+$"min_add"*60,"HH:mm:ss"))
                         .withColumn("Delta",minute($"Dtime")-minute($"TimeH"))
                         .withColumn("CountDelta",count("Delta") over w_deltaNb )
                         .where($"CountDelta"===24)
                         .drop("minute","new_min","min_add","Dtime")
                         .sort($"CallSign" .asc,$"DateH" .asc, $"Time")

// On ne conserve qu'un relevé horaire de 24 heures lorsqu'il y en a plusieurs sur une même journée
val hourly2 = hourly.withColumn("RgDelta", rank() over w_deltaRg )
                    .where($"RgDelta" ===1)               
                    .withColumn("_t_", hour($"TimeH"))
                    //.where($"_t_"<=2)
                    .drop("Delta","CountDelta","RgDelta")
                    //.sort($"CallSign" .asc,$"DateH" .asc, $"Time")

print("Nb stations avec relevé météo sur 24h : ")
println(hourly2.count())

hourly2.select("CallSign","Date","DateH","Time","TimeH","_t_")
       .where($"CallSign"==="DTW").show(25, false)

Nb stations avec relevé météo sur 24h : 204600
+--------+--------+----------+----+--------+---+
|CallSign|Date    |DateH     |Time|TimeH   |_t_|
+--------+--------+----------+----+--------+---+
|DTW     |20131101|2013-11-01|0053|00:00:00|0  |
|DTW     |20131101|2013-11-01|0153|01:00:00|1  |
|DTW     |20131101|2013-11-01|0253|02:00:00|2  |
|DTW     |20131101|2013-11-01|0353|03:00:00|3  |
|DTW     |20131101|2013-11-01|0453|04:00:00|4  |
|DTW     |20131101|2013-11-01|0553|05:00:00|5  |
|DTW     |20131101|2013-11-01|0653|06:00:00|6  |
|DTW     |20131101|2013-11-01|0753|07:00:00|7  |
|DTW     |20131101|2013-11-01|0853|08:00:00|8  |
|DTW     |20131101|2013-11-01|0953|09:00:00|9  |
|DTW     |20131101|2013-11-01|1053|10:00:00|10 |
|DTW     |20131101|2013-11-01|1153|11:00:00|11 |
|DTW     |20131101|2013-11-01|1253|12:00:00|12 |
|DTW     |20131101|2013-11-01|1353|13:00:00|13 |
|DTW     |20131101|2013-11-01|1453|14:00:00|14 |
|DTW     |20131101|2013-11-01|1553|15:00:00|15 |
|DTW     |20131101|201

import org.apache.spark.sql.functions.unix_timestamp
import org.apache.spark.sql.functions.{to_date, to_timestamp}
import org.apache.spark.sql.functions.{hour, minute, second}
import org.apache.spark.sql.expressions.Window
w_deltaNb: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@5c093d5c
w_deltaRg: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7cbf51dc
hourly: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string, State: string ... 12 more fields]
hourly2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string, State: string ... 11 more fields]


### (ii) Création de la colonne Weather_h reprenant sous forme d'un Array les 24 relevés météo
WOT_201311 est une table de niveau station / Date en heure pleine (DateH) donnant les infos météo sous la forme d'un Array. 3 colonnes : CallSign, DateH et Weather_h
* La colonne SkyCondition est splitée en plusiers colonnes correspondant aux différentes couches nuageuses mesurées (jusque 3 colonnes pour une station automatique). Chaque colonne correspondant à une couche nuageuse est elle-même splitée entre le code de 3 lettres (CLR, OVC, SCT...) et la hauteur de la couche. Le code de couverture nuageuse est ensuite indexé.
* Les infos météo de type numérique (windspeed, station pressure) sont transformées en Double


In [58]:
// Transformation of SkyCondition column

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Column

val skyCond = hourly2.withColumn("SkyC", split($"SkyCondition", " "))

// Function to split code part and layer high from SkyCondition observation
def splitCode(DFrame : DataFrame, df_col : Column): DataFrame = {
    val fields = List( (df_col.toString+"_code","CLR"),(df_col.toString+"_high","000") )

    fields.foldLeft( (DFrame,1) ){ 
    (tempdf, field) =>
    val newdf = tempdf._1.withColumn(field._1, when(df_col ==="CLR","000").
                                               otherwise(substring(df_col, tempdf._2,3)))
                         .withColumn(field._1, when(col(field._1) .isNull,"000").
                                               otherwise(col(field._1)))    
    (newdf, tempdf._2 + 3)
  }._1
}

// first split 
val numCols = skyCond
              .withColumn("SkyC_size", size($"SkyC"))
              .agg(max($"SkyC_size"))
              .head()
              .getInt(0)
    
val skyCond_first_split = skyCond.select(col("*") +:
                            (1 until numCols + 1).map(i => $"SkyC".getItem(i-1).as(s"SC_$i")): _*)
                          .withColumn("NumLayer",size($"SkyC"))
                          .drop($"SkyC")

// second split, colonnes "SC_i", i=1 to numCols
def listSC = for (i <- (1 to numCols).toList) yield ("SC_"+i)

var resultDF = skyCond_first_split
for (x <- listSC) {
    resultDF = splitCode(resultDF,col(x))
}
val skyCond_second_split = resultDF.drop(listSC:_*)

skyCond_second_split.select("CallSign","Date","Time","SkyCondition", "NumLayer","SC_1_code", "SC_1_high", 
                        "SC_2_code", "SC_2_high", "SC_3_code", "SC_3_high").show(10)

+--------+--------+----+--------------------+--------+---------+---------+---------+---------+---------+---------+
|CallSign|    Date|Time|        SkyCondition|NumLayer|SC_1_code|SC_1_high|SC_2_code|SC_2_high|SC_3_code|SC_3_high|
+--------+--------+----+--------------------+--------+---------+---------+---------+---------+---------+---------+
|     ABE|20131101|0051|              FEW016|       1|      FEW|      016|      000|      000|      000|      000|
|     ABE|20131101|0151|BKN033 BKN047 OVC055|       3|      BKN|      033|      BKN|      047|      OVC|      055|
|     ABE|20131101|0251|              OVC028|       1|      OVC|      028|      000|      000|      000|      000|
|     ABE|20131101|0351|BKN023 BKN033 OVC095|       3|      BKN|      023|      BKN|      033|      OVC|      095|
|     ABE|20131101|0451|FEW022 BKN050 OVC100|       3|      FEW|      022|      BKN|      050|      OVC|      100|
|     ABE|20131101|0551|SCT023 BKN029 OVC037|       3|      SCT|      023|      

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Column
skyCond: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 12 more fields]
splitCode: (DFrame: org.apache.spark.sql.DataFrame, df_col: org.apache.spark.sql.Column)org.apache.spark.sql.DataFrame
numCols: Int = 3
skyCond_first_split: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 15 more fields]
listSC: List[String]
resultDF: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 21 more fields]
skyCond_second_split: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 18 more fields]


In [59]:
// string index the categorical attributes SC_i_code

import org.apache.spark.ml.feature.StringIndexer
import  org.apache.spark.ml.Pipeline 

val skyCond_index = skyCond_second_split.select("CallSign","Date","Time","SC_1_code","SC_2_code","SC_3_code")

val SCindexers = skyCond_second_split.columns.filter(_.contains("code")).map
{field => 
    new StringIndexer()
    .setInputCol(field)
    .setStringOrderType("alphabetDesc") 
    .setOutputCol(field+"_ix") 
}

val pipeline = new Pipeline()
                  .setStages(SCindexers)
val skyCond_splitted = pipeline.fit(skyCond_second_split).transform(skyCond_second_split)

skyCond_splitted.select("CallSign","Date","Time","NumLayer","SC_1_code","SC_1_code_ix","SC_2_code","SC_2_code_ix",
                       "SC_3_code","SC_3_code_ix").show(5)

+--------+--------+----+--------+---------+------------+---------+------------+---------+------------+
|CallSign|    Date|Time|NumLayer|SC_1_code|SC_1_code_ix|SC_2_code|SC_2_code_ix|SC_3_code|SC_3_code_ix|
+--------+--------+----+--------+---------+------------+---------+------------+---------+------------+
|     ABE|20131101|0051|       1|      FEW|         4.0|      000|         5.0|      000|         5.0|
|     ABE|20131101|0151|       3|      BKN|         6.0|      BKN|         4.0|      OVC|         1.0|
|     ABE|20131101|0251|       1|      OVC|         2.0|      000|         5.0|      000|         5.0|
|     ABE|20131101|0351|       3|      BKN|         6.0|      BKN|         4.0|      OVC|         1.0|
|     ABE|20131101|0451|       3|      FEW|         4.0|      BKN|         4.0|      OVC|         1.0|
+--------+--------+----+--------+---------+------------+---------+------------+---------+------------+
only showing top 5 rows



import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.Pipeline
skyCond_index: org.apache.spark.sql.DataFrame = [CallSign: string, Date: string ... 4 more fields]
SCindexers: Array[org.apache.spark.ml.feature.StringIndexer] = Array(strIdx_4f21c8d19b53, strIdx_5bfe3a018d5f, strIdx_e2a6dc55ad51)
pipeline: org.apache.spark.ml.Pipeline = pipeline_a80c00dbdf9d
skyCond_splitted: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 21 more fields]


In [75]:
// Tranformation of numeric measures from string en double

def listSC_high = for (i <- (1 to numCols).toList) yield ("SC_"+i+"_high")
val listVars = List("WindSpeed", "StationPressure","Tmax","Tmin","AvgTemp", "NumLayer")
val fields = listVars ++ listSC_high


val hourly2_double = fields.foldLeft(skyCond_splitted){ 
                     (tempdf, colName) => tempdf.withColumn(colName, col(colName).cast("Double")) }


// vectorization of weather observation in a unique Weather column : IMPOSSIBLE WITH COLLECT_LIST
/*import org.apache.spark.ml.feature.VectorAssembler

def listSC_code = for (i <- (1 to numCols).toList) yield ("SC_"+i+"_code_ix")
val cols = (listSC_code ++ fields).toArray

val assembler = new VectorAssembler()
  .setInputCols(cols)
  .setOutputCol("Weather")

val hourly3 = assembler.transform(hourly2_double)
                       .select("CallSign","DateH", "Time","TimeH","_t_","Weather")
                       .sort($"CallSign" .asc,$"DateH" .asc, $"Time")
*/

// Concatenation in a struct 
/*import org.apache.spark.sql.functions.struct
import org.apache.spark.sql.Column

def listSC_code = for (i <- (1 to numCols).toList) yield ("SC_"+i+"_code_ix")
val needed_cols : List[String] = (listSC_code ++ fields)

// Varable Weather regroupant, pour chaque heure, les infos météo retenues
val hourly3 = hourly2_double.withColumn("Weather", struct(needed_cols.map(col): _*))
                            .select("CallSign","DateH", "Time","TimeH","_t_","Weather")
                            .sort($"CallSign" .asc,$"DateH" .asc, $"Time")
*/

// Concatenation in an array
import org.apache.spark.sql.functions.{col,concat_ws}

def listSC_code = for (i <- (1 to numCols).toList) yield ("SC_"+i+"_code_ix")
val needed_cols : List[String] = (listSC_code ++ fields)

val hourly3 = hourly2_double.withColumn("Weather",array(needed_cols.map(col): _*))
                            .select("CallSign","DateH", "Time","TimeH","_t_","Weather")
                            .sort($"CallSign" .asc,$"DateH" .asc, $"Time")
hourly3.show(false)
hourly3.printSchema

+--------+----------+----+--------+---+----------------------------------------------------------------------+
|CallSign|DateH     |Time|TimeH   |_t_|Weather                                                               |
+--------+----------+----+--------+---+----------------------------------------------------------------------+
|ABE     |2013-11-01|0051|00:00:00|0  |[4.0, 5.0, 5.0, 13.0, 29.3, 70.0, 47.0, 40.5, 1.0, 16.0, 0.0, 0.0]    |
|ABE     |2013-11-01|0151|01:00:00|1  |[6.0, 4.0, 1.0, 11.0, 29.27, 70.0, 47.0, 40.5, 3.0, 33.0, 47.0, 55.0] |
|ABE     |2013-11-01|0251|02:00:00|2  |[2.0, 5.0, 5.0, 15.0, 29.23, 70.0, 47.0, 40.5, 1.0, 28.0, 0.0, 0.0]   |
|ABE     |2013-11-01|0351|03:00:00|3  |[6.0, 4.0, 1.0, 15.0, 29.19, 70.0, 47.0, 40.5, 3.0, 23.0, 33.0, 95.0] |
|ABE     |2013-11-01|0451|04:00:00|4  |[4.0, 4.0, 1.0, 16.0, 29.15, 70.0, 47.0, 40.5, 3.0, 22.0, 50.0, 100.0]|
|ABE     |2013-11-01|0551|05:00:00|5  |[1.0, 4.0, 1.0, 20.0, 29.13, 70.0, 47.0, 40.5, 3.0, 23.0, 29.0, 37.0] |
|

listSC_high: List[String]
listVars: List[String] = List(WindSpeed, StationPressure, Tmax, Tmin, AvgTemp, NumLayer)
fields: List[String] = List(WindSpeed, StationPressure, Tmax, Tmin, AvgTemp, NumLayer, SC_1_high, SC_2_high, SC_3_high)
hourly2_double: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 21 more fields]
import org.apache.spark.sql.functions.{col, concat_ws}
listSC_code: List[String]
needed_cols: List[String] = List(SC_1_code_ix, SC_2_code_ix, SC_3_code_ix, WindSpeed, StationPressure, Tmax, Tmin, AvgTemp, NumLayer, SC_1_high, SC_2_high, SC_3_high)
hourly3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string, DateH: date ... 4 more fields]


In [84]:
// Table Variable Weather_h
val WOT_201311 = hourly3
  .groupBy($"CallSign",$"DateH")
  .agg(collect_list("Weather").as("Weather_h"))


println(s"Nb stations / Date : ${WOT_201311.count()}")
WOT_201311.show(1, false) 
WOT_201311.printSchema

//---------------------------------------------------------------------------------------
// Remark : to flatten the array of array column to array : à utiliser en étape 5/5 du 4.
val WOT_201311_flattened = WOT_201311.withColumn("Weather_hf",flatten($"Weather_h"))
                                     .drop("Weather_h")

// ... puis en Vector
import org.apache.spark.ml.linalg._

val convertUDF = udf((array : Seq[Double]) => {
  Vectors.dense(array.toArray)
})

val WOT_201311_vector = WOT_201311_flattened.withColumn("Weather_hv", convertUDF($"Weather_hf"))
                                            .drop("Weather_hf")
WOT_201311_vector.printSchema


+--------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

WOT_201311: org.apache.spark.sql.DataFrame = [CallSign: string, DateH: date ... 1 more field]
WOT_201311_flattened: org.apache.spark.sql.DataFrame = [CallSign: string, DateH: date ... 1 more field]
import org.apache.spark.ml.linalg._
convertUDF: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5305/1650481884@1d3c0bf5,org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7,List(Some(class[value[0]: array<double>])),None,true,true)
WOT_201311_vector: org.apache.spark.sql.DataFrame = [CallSign: string, DateH: date ... 1 more field]


# 3. Préparation des données de vols => Flight Table (FT)
La Table des données de vols contient les informations sur les vols et est limitée aux vols non détournés (diverted) ou annulés (canceled) 

Examen des variables :
* dep_time et arr_time : heure de départ effective

* crs_dep_time et crs_arr_time : heure de départ et d'arrivée prévus
    * Computerized reservation system (CRS) departure/arrival time is the scheduled departure/arrival time of the flight,
    * wheels off is the time when the wheels of the aircraftleave the ground at the origin airport, and 
    * wheels on is the time when the wheels of the aircraft touch theground at the destination airport.

* The departure delay of  an  aircraft  is  the  difference  between  the  actual departure  time  and  the  CRS  departure time of the flight. 

* Arrival delay equals actual arrival time minusthe scheduled arrival time.

* "dep_time_blk" : créneau horaire. ex. crs_dep_time=1301 => 1300-1359

* différence entre dep_delay et dep_delay_new ? ici dep_delay peut être négatif et traduire un vol en avance


## Etape 1/2 - Limitation du périmètre des vols
* Limitation aux vols pours lesquels correspond une sation météo au départ ou à l'arrivée
* Limitation aux vols non annulés (canceled) et détournés (diverted)
* Récupération, à partir des données de stations (data_station), de la TimeZone pour les aéroports de départ et d'arrivée et calcul du Time_lag entre aéroports
* Création d'une clé entre les vols au départ et les vols à l'arrivée

In [4]:
// novembre 2013

val data_vols1 = spark.read.format("csv").option("header", "true")
                      .load(pathA+date1+"_T_ONTIME_REPORTING.csv")
                      .where( ($"Diverted" !== 1) || ($"Cancelled" != 1) )
                      .filter("tail_num is not null")
                      .filter("dep_time is not null")

print("Nb de vols non annulés ou non détournés : ")
println(data_vols1.count())


// Limitations aux vols pour lesquels une station météo est trouvée pour le départ ou l'arrivé
val data_vols2  = data_vols1.join(data_station.select("CallSign","TimeZone"), data_vols1("origin") === data_station("CallSign"),"inner")
                           .drop("CallSign")
                           .withColumnRenamed("TimeZone", "dep_TZone")
                           .join(data_station.select("CallSign","TimeZone"), data_vols1("dest") === data_station("CallSign"),"inner")
                           .drop("CallSign")
                           .withColumnRenamed("TimeZone", "arr_TZone")
                           .withColumn("Time_lag",$"arr_TZone"-$"dep_TZone")
                           .withColumn("Cle", concat($"fl_date", $"tail_num",$"origin",$"dep_time",$"dest",$"arr_time"))

print("Nb de vols non annulés ou non détournés avec station météo : ")
println(data_vols2.count())
data_vols2.select("cle","fl_date","origin", "tail_num", "dep_time",
                                   "dest","arr_time","ACTUAL_ELAPSED_TIME", "Time_lag").show(5)

// Table des vols
val FT_201311 = data_vols2.select("cle","fl_date","origin", "tail_num", "dest","dep_time","crs_dep_time","dep_delay",
                                  "dep_delay_new","dep_del15", "dep_delay_group",
                                  "arr_time","crs_arr_time","arr_delay","arr_delay_new","arr_del15",
                                  "ACTUAL_ELAPSED_TIME","distance","Time_lag")

Nb de vols non annulés ou non détournés : 498420
Nb de vols non annulés ou non détournés avec station météo : 497177
+--------------------+----------+------+--------+--------+----+--------+-------------------+--------+
|                 cle|   fl_date|origin|tail_num|dep_time|dest|arr_time|ACTUAL_ELAPSED_TIME|Time_lag|
+--------------------+----------+------+--------+--------+----+--------+-------------------+--------+
|2013-11-01N8747BD...|2013-11-01|   DTW|  N8747B|    2126| BGM|    2238|              72.00|     0.0|
|2013-11-03N8883ED...|2013-11-03|   DTW|  N8883E|    2136| BGM|    2251|              75.00|     0.0|
|2013-11-04N8709AD...|2013-11-04|   DTW|  N8709A|    2145| BGM|    2259|              74.00|     0.0|
|2013-11-05N8918BD...|2013-11-05|   DTW|  N8918B|    2136| BGM|    2245|              69.00|     0.0|
|2013-11-06N8914AD...|2013-11-06|   DTW|  N8914A|    2133| BGM|    2242|              69.00|     0.0|
+--------------------+----------+------+--------+--------+----+----

data_vols1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [YEAR: string, QUARTER: string ... 60 more fields]
data_vols2: org.apache.spark.sql.DataFrame = [YEAR: string, QUARTER: string ... 64 more fields]
FT_201311: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 17 more fields]


## Etape 2/2 - Préparation de la table des vols avant jointure avec les données météo => flightD et flightA

En vue du croisement avec les données météo sur les douze dernières heures, deux tables sont créés l'une pour les vols au départ (flightD), l'autre pour les vols à l'arrivée (flightA)

### (i) Gestion des dates pour les vols

La gestion des dates et des heures concernent à la fois les informations au départ et les informations à l'arrivée.
La date d'arrivée n'est pas renseignée. Elle est reconstituée (ADate) à partir de la date de départ, de l'heure de départ, de la durée totale du vol (actual_elapsed_time) et du time_lag entre les deux aéroports.
Afin de pouvoir croiser les données de vols avec les relevés météo, des heures de référence pour le départ et l'arrivée (DTimeF et ATimeF) sont créées : elles correspondent à l'heure pleine immédiatement inférieure.

* Création de la variable DDate correspondant à fl_date en format date

* Création de la variable ADate correspondant à la date d'arrivée 

* Création de la variable DDateF (departure) et ADateF (arrival) correspondant à la date de référence (heure pleine immédiatement inférieure) pour le rapprochement avec les relevés météo
    * si l'horaire de vol est après 10h, DateF = fl_date ;
    * pour les vols dont une partie des douze dernières heures couvrent le jour précédent, nous créons une nouvelle observation avec DateF au jour précédent (passage de 502507 obs à 706897 obs).

* Création de la variable DTime / ATime correspondant à l'heure de départ et DTimeF / ATimeF correspondant à l'heure pleine la plus proche de l'heure de départ

* La variable _d_ (departure) et _a_ (arrival) correspond au numéro de l'heure de départ et _d0_ / _a0_ au numéro de l'heure la plus petite du jour de départ(DDateF) ou d'arrivée (ADateF) pour laquelle un relevé météo est à récupérer :
    * si l'heure de départ de référence est DTimeF=12:00, pour ce jour : _d_=12 et _d0_=1 et les relevés horaires 1, ..., 12 seront nécessaires (idem pour _a_ et _a0_);
    * si l'heure de départ de référence est DTimeF=08:00, pour ce jour : le numéro de l'heure de départ est _d_ = 8 et _d0_=0 et les relevés horaires 0,1,...,8 seront nécessaires (idem pour _a_ et _a0_). A noter que dans ce cas, il sera nécessaire de créer un enregistrement complémentaire avec la date du jour précédent et _d_=23 et _d0_=21 (relevés 21,22,23) seront nécessaires ( voir (ii) ).

In [13]:
import org.apache.spark.sql.functions.unix_timestamp
import org.apache.spark.sql.functions.{to_date, to_timestamp}
import org.apache.spark.sql.functions.{hour,minute,second}

import org.apache.spark.sql.expressions.Window

// gestion des dates pour le départ : DDate, DTime, DDateF, DTimeF, _d_ et _d0_
val data_vols3 = data_vols2.select("cle","fl_date","origin", "tail_num", "dep_time",
                                   "dest","arr_time","ACTUAL_ELAPSED_TIME", "Time_lag")
                           .withColumn("DDate",to_date(unix_timestamp($"fl_date","yyyy-MM-dd").cast("timestamp")))
                           .withColumn("DDateF",to_date(unix_timestamp($"fl_date","yyyy-MM-dd").cast("timestamp")))
                           .withColumn("DTime",from_unixtime(unix_timestamp($"dep_time","HHmm"),"HH:mm:ss"))
                           .withColumn("Dminute", minute($"DTime"))
                           .withColumn("Dnew_min", (floor($"Dminute"/60))*60)
                           .withColumn("Dmin_sub", $"Dnew_min"- $"Dminute")
                           .withColumn("DTimeF", from_unixtime(unix_timestamp($"DTime","HH:mm:ss")+$"Dmin_sub"*60,"HH:mm:ss"))
                           .withColumn("_d_", hour($"DTimeF"))
                           .withColumn("_d0_", when($"_d_" > 11, $"_d_"-11).otherwise(0))
                           .withColumn("_ddb_",when($"_d_" <11, 23))            // renseigné si chevauchement sur deux dates
                           .withColumn("_ddb0_", when($"_d_" <11, $"_d_"+13))   // renseigné si chevauchement sur deux dates           
                           .drop("Dminute","Dnew_min","Dmin_sub")

println("Extract de vols au départ : ")
data_vols3.select("DDate","dep_time","DTime","DDateF","DTimeF","_d_", "_d0_", "_ddb_", "_ddb0_").show(5, false)

// gestion des dates pour l'arrivée : ADate, ATime, ADateF, ATimeF, _a_ et _a0_
val data_vols4 = data_vols3.withColumn("ATime",from_unixtime(unix_timestamp($"arr_time","HHmm"),"HH:mm:ss"))
                           .withColumn("Time0",from_unixtime(unix_timestamp(lit("2359"),"HHmm"),"HH:mm:ss"))
                           .withColumn("Timeto0",round( (unix_timestamp($"Time0","HH:mm:ss") - 
                                                 unix_timestamp($"DTime","HH:mm:ss")+1)/60, 2))
                           .withColumn("CORR_ELAPSED_TIME",$"ACTUAL_ELAPSED_TIME" + $"Time_lag"*60)
                           .withColumn("ADate", when($"CORR_ELAPSED_TIME" > $"TimeTo0", date_add(to_date($"DDate","yyyy-MM-dd"),+1))
                                                .otherwise($"DDate"))
                           .withColumn("ADateF",$"ADate")
                           .withColumn("Aminute", minute($"ATime"))
                           .withColumn("Anew_min", (floor($"Aminute"/60))*60)
                           .withColumn("Amin_sub", $"Anew_min"- $"Aminute")
                           .withColumn("ATimeF", from_unixtime(unix_timestamp($"ATime","HH:mm:ss")+$"Amin_sub"*60,"HH:mm:ss"))
                           .withColumn("_a_", hour($"ATimeF"))
                           .withColumn("_a0_", when($"_a_" > 11, $"_a_"-11).otherwise(0))
                           .withColumn("_adb_",when($"_a_" <11, 23))
                           .withColumn("_adb0_", when($"_a_" <11, $"_a_"+13))            
                           .drop("Aminute","Anew_min","Amin_sub")

println("Extract de vols à l'arrivée : ")
data_vols4.select("DDate","ADate","ATime","ADateF","ATimeF","_a_", "_a0_", "_adb_", "_adb0_").show(5, false)

Extract de vols au départ
+----------+--------+--------+----------+--------+---+----+-----+------+
|DDate     |dep_time|DTime   |DDateF    |DTimeF  |_d_|_d0_|_ddb_|_ddb0_|
+----------+--------+--------+----------+--------+---+----+-----+------+
|2013-11-01|2126    |21:26:00|2013-11-01|21:00:00|21 |10  |null |null  |
|2013-11-03|2136    |21:36:00|2013-11-03|21:00:00|21 |10  |null |null  |
|2013-11-04|2145    |21:45:00|2013-11-04|21:00:00|21 |10  |null |null  |
|2013-11-05|2136    |21:36:00|2013-11-05|21:00:00|21 |10  |null |null  |
|2013-11-06|2133    |21:33:00|2013-11-06|21:00:00|21 |10  |null |null  |
+----------+--------+--------+----------+--------+---+----+-----+------+
only showing top 5 rows

Extract de vols à l'arrivée
+----------+----------+--------+----------+--------+---+----+-----+------+
|DDate     |ADate     |ATime   |ADateF    |ATimeF  |_a_|_a0_|_adb_|_adb0_|
+----------+----------+--------+----------+--------+---+----+-----+------+
|2013-11-01|2013-11-01|22:38:00|2013-11

import org.apache.spark.sql.functions.unix_timestamp
import org.apache.spark.sql.functions.{to_date, to_timestamp}
import org.apache.spark.sql.functions.{hour, minute, second}
import org.apache.spark.sql.expressions.Window
data_vols3: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 15 more fields]
data_vols4: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 26 more fields]


### (ii) Création des enregistrements supplémentaires
Lorque les douzes heures qui précèdent l'heure de départ ou l'heure d'arrivée des vols chevauchent le jour qui précède le départ ou l'arrivée, de nouveaux enregistrements sont créés avec comme DDateF ou ADateF la date du jours qui précède. Ces enregistrements permettront de récupérer les données météo de la veille.
Deux tables sont créées :
* flightD pour les informations de départ ;
* filghtA pour les informations d'arrivée.

In [14]:
// Cas des vols au départ
val vols_dep = data_vols4.select("cle","fl_date","DDate","DDateF","origin","tail_num","dep_time","DTime",
                                   "DTimeF","_d_","_d0_", "_ddb_", "_ddb0_") 

// Nouvel enregistrement pour les vols dont l'heure de départ est inférieure à 12h (DDateF moins un jour)
val vols_dep_new = vols_dep.where($"_d_"<11)
                           .withColumn("_d0_",$"_ddb0_")
                           .withColumn("_d_",$"_ddb_")
                           .withColumn("DDateF1", date_add(to_date($"DDateF","yyyy-MM-dd"),-1))
                           .withColumn("DDateF",$"DDateF1")
                           .drop("DDateF1")          

val flightD = vols_dep.union(vols_dep_new)
                      .select("cle","fl_date","DDate","DDateF","origin","tail_num","dep_time","DTime",
                              "DTimeF","_d_","_d0_")

print("Table flightD. Nb vols/Date au départ après enrichissement : ")
println(flightD.count())

println("Extract de vols au départ, aéroport ADK : ")
flightD.where($"origin"==="ADK").drop("cle").show(5, false)
println("Extract de vols au départ, nouveaux enregistrements : ")
flightD.where($"DDate" !== $"DDateF").drop("cle").show(5, false)

Nb vols/Date au départ après enrichissement : 668074
+----------+----------+----------+------+--------+--------+--------+--------+---+----+
|fl_date   |DDate     |DDateF    |origin|tail_num|dep_time|DTime   |DTimeF  |_d_|_d0_|
+----------+----------+----------+------+--------+--------+--------+--------+---+----+
|2013-11-28|2013-11-28|2013-11-28|ADK   |N764AS  |1713    |17:13:00|17:00:00|17 |6   |
|2013-11-21|2013-11-21|2013-11-21|ADK   |N713AS  |1733    |17:33:00|17:00:00|17 |6   |
|2013-11-24|2013-11-24|2013-11-24|ADK   |N799AS  |1753    |17:53:00|17:00:00|17 |6   |
|2013-11-03|2013-11-03|2013-11-03|ADK   |N771AS  |1639    |16:39:00|16:00:00|16 |5   |
|2013-11-10|2013-11-10|2013-11-10|ADK   |N799AS  |1815    |18:15:00|18:00:00|18 |7   |
+----------+----------+----------+------+--------+--------+--------+--------+---+----+
only showing top 5 rows

+----------+----------+----------+------+--------+--------+--------+--------+---+----+
|fl_date   |DDate     |DDateF    |origin|tail_num|de

vols_dep: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 11 more fields]
vols_dep_new: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 11 more fields]
flightD: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 9 more fields]


In [15]:
// Cas des vols à l'arrivée

val vols_arr = data_vols4.select("cle","fl_date","ATime","ADate","ADateF","ATimeF","dest","tail_num", "arr_time",
                                 "_a_", "_a0_", "_adb_", "_adb0_")

// Nouvel enregistrement pour les vols dont l'heure d'arrivée est inférieure à 12h (ADateF moins un jour)
val vols_arr_new = vols_arr.where($"_a_"<11)
                           .withColumn("_a0_",$"_adb0_")
                           .withColumn("_a_",$"_adb_")
                           .withColumn("ADateF1", date_add(to_date($"ADateF","yyyy-MM-dd"),-1))
                           .withColumn("ADateF",$"ADateF1")
                           .drop("ADateF1")          

val flightA = vols_arr.union(vols_arr_new)
                      .select("cle","fl_date","ADate","ADateF","dest","tail_num","arr_time","ATime",
                              "ATimeF","_a_","_a0_")

print("Table flightA. Nb vols/Date à l'arrivée après enrichissement : ")
println(flightA.count())
println("Extract de vols à l'arrivée, aéroport ADK : ")
flightA.where($"dest"==="ADK").drop("cle").show(5, false)
println("Extract de vols à l'arrivée, nouveaux enregistrements : ")
flightA.where($"ADate" !== $"ADateF").drop("cle").show(5, false)

Nb vols/Date à l'arrivée après enrichissement : 610650
Extract de vols à l'arrivée, aéroport ADK : 
+----------+----------+----------+----+--------+--------+--------+--------+---+----+
|fl_date   |ADate     |ADateF    |dest|tail_num|arr_time|ATime   |ATimeF  |_a_|_a0_|
+----------+----------+----------+----+--------+--------+--------+--------+---+----+
|2013-11-28|2013-11-28|2013-11-28|ADK |N764AS  |1606    |16:06:00|16:00:00|16 |5   |
|2013-11-21|2013-11-21|2013-11-21|ADK |N713AS  |1549    |15:49:00|15:00:00|15 |4   |
|2013-11-24|2013-11-24|2013-11-24|ADK |N799AS  |1645    |16:45:00|16:00:00|16 |5   |
|2013-11-03|2013-11-03|2013-11-03|ADK |N771AS  |1544    |15:44:00|15:00:00|15 |4   |
|2013-11-10|2013-11-10|2013-11-10|ADK |N799AS  |1633    |16:33:00|16:00:00|16 |5   |
+----------+----------+----------+----+--------+--------+--------+--------+---+----+
only showing top 5 rows

Extract de vols à l'arrivée, nouveaux enregistrements : 
+----------+----------+----------+----+--------+-----

vols_arr: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 11 more fields]
vols_arr_new: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 11 more fields]
flightA: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 9 more fields]


# 4. Table des données de vols enrichie des données météo => JT_201311
Construction du dataframe reprenant les vols avec un colonne Weather_d donnant les relevés météo des douze dernières heures précédant l'heure de départ et Weather_a donnant les relevés météo des douze heures précédant l'heure d'arrivée.

## Etape 1/5 : croisement entre le dataframe des vols et le dataframe reprenant le relevé météo par heure

ATTENTION, le croisement correspondant à la première date du jour du mois n'est pas géré ici.

In [19]:
val df_hourly = WOT_201311
df_hourly.printSchema
println(df_hourly.count())

root
 |-- CallSign: string (nullable = true)
 |-- DateH: date (nullable = true)
 |-- Weather_h: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- SkyCondition: string (nullable = true)
 |    |    |-- WindSpeed: string (nullable = true)
 |    |    |-- StationPressure: string (nullable = true)
 |    |    |-- Tmax: string (nullable = true)
 |    |    |-- Tmin: string (nullable = true)
 |    |    |-- AvgTemp: string (nullable = true)

8525


df_hourly: org.apache.spark.sql.DataFrame = [CallSign: string, DateH: date ... 1 more field]


In [17]:
// Cas des vols au départ

val flightD_H = flightD.join(df_hourly, flightD("origin") === df_hourly("CallSign") &&
                                         flightD("DDateF") === df_hourly("DateH"), "inner" )
                       .withColumnRenamed("Weather_h","Weather_d")
                       .drop("DateH", "CallSign","Date")
                       .sort($"origin" .asc, $"DDateF" .asc, $"tail_num" .asc, $"DTime" .asc)

flightD_H.select("fl_date","DDateF","origin", "tail_num","DTime","_d_","_d0_","Weather_d").show(1,false)
print("Table flightD_H. Nb vols/Date au départ avec relevé météo : ")
println(flightD_H.count())

+----------+----------+------+--------+--------+---+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|fl_date   |DDateF    |origin|t

flightD_H: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cle: string, fl_date: string ... 10 more fields]


In [18]:
// Cas des vols à l'arrivée

val flightA_H = flightA.join(df_hourly, flightA("dest") === df_hourly("CallSign") &&
                                        flightA("ADateF") === df_hourly("DateH"), "inner" )
                       .withColumnRenamed("Weather_h","Weather_a")
                       .drop("DateH", "CallSign","Date")
                       .sort($"dest" .asc, $"ADateF" .asc, $"tail_num" .asc, $"ATime" .asc)

flightA_H.select("fl_date","ADateF","dest", "tail_num","ATime","_a_","_a0_","Weather_a").show(1,false)
print("Table flightA_H. Nb vols/Date à l'arrivée avec relevé météo : ")
println(flightA_H.count())

+----------+----------+----+--------+--------+---+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|fl_date   |ADateF    |dest|tail_

flightA_H: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cle: string, fl_date: string ... 10 more fields]


## Etape 2/5 : élimination des relevés météo qui ne correspondant pas aux douze dernières heures précédant le départ ou l'arrivée du vol
Pour éliminer les relevés météo non nécessaires, les relevés horaires sont repositionnés en ligne à ligne (explode).
Seuls les vols avec 12 relevés horaires disponibles sont retenus (ce qui suppose que les relevés horaires existent pour ce vol sur deux dates lorsque les vols partent ou arrivent après 10h du matin).

In [59]:
import org.apache.spark.sql.functions.{explode, flatten}

// cas des vols au départ

val w_hourD = Window
                .partitionBy("origin","DDate","DDateF","tail_num","DTime")
                .orderBy($"DTime" .asc)
val w_obsD = Window
                .partitionBy("origin","DDate","tail_num","DTime")
                .orderBy($"DTime" .asc)

val flightD_H_explode = flightD_H.withColumn("Weather_t",explode($"Weather_d"))
                                 .withColumn("rn", row_number over w_hourD)
                                 .withColumn("_t_", $"rn"-1 )
                                 .where($"_t_">=$"_d0_" && $"_t_"<= $"_d_")  // sélection des relevés horaires
                                 .withColumn("obs", count("DTime") over w_obsD)
                                 .where($"obs" === 12)                       // limitation à 12 relevés horaires
                                 .select("cle","origin","dep_time","fl_date","DDate","DDateF","tail_num","DTime","_d_","_d0_","_t_","obs","Weather_t")
                                 .sort($"origin" .asc, $"DDate" .asc, $"tail_num" .asc, $"DTime" .asc, $"_t_" .asc)

print("Table flightD_H_explode. Nb vols/Dates au départ avec 12 relevés horaires : ")
println(flightD_H_explode.count())

Table flightD_H_explode. Nb vols/Dates au départ avec 12 relevés horaires : 5535744


w_hourD: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3e85b89f
w_obsD: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7f9d0aea
flightD_H_explode: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cle: string, origin: string ... 11 more fields]


In [36]:
// cas des vols à l'arrivée

val w_hourA = Window
                .partitionBy("dest","ADate","ADateF","tail_num","ATime")
                .orderBy($"ATime" .asc)
val w_obsA = Window
                .partitionBy("dest","ADate","tail_num","ATime")
                .orderBy($"ATime" .asc)

val flightA_H_explode = flightA_H.withColumn("Weather_t",explode($"Weather_a"))
                                 .withColumn("rn", row_number over w_hourA)
                                 .withColumn("_t_", $"rn"-1 )
                                 .where($"_t_">=$"_a0_" && $"_t_"<= $"_a_")
                                 .withColumn("obs", count("ATime") over w_obsA)
                                 .where($"obs"===12)
                                 .select("cle","dest","arr_time","fl_date","ADate","ADateF","tail_num","ATime","_a_","_a0_","_t_","obs","Weather_t")
                                 .sort($"dest" .asc, $"ADate" .asc, $"tail_num" .asc, $"ATime" .asc, $"_t_" .asc)

print("Table flightA_H_explode. Nb vols/Dates au départ avec 12 relevés horaires : ")
println(flightA_H_explode.count())

Table flightA_H_explode. Nb vols/Dates au départ avec 12 relevés horaires : 5557104


w_hourA: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@34366af9
w_obsA: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@25a67762
flightA_H_explode: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cle: string, dest: string ... 11 more fields]


## Etape 3/5 : vols avec leurs 12 relevés météo

In [38]:
flightD_H_explode.printSchema

root
 |-- cle: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- fl_date: string (nullable = true)
 |-- DDate: date (nullable = true)
 |-- DDateF: date (nullable = true)
 |-- tail_num: string (nullable = true)
 |-- DTime: string (nullable = true)
 |-- _d_: integer (nullable = true)
 |-- _d0_: integer (nullable = true)
 |-- _t_: integer (nullable = true)
 |-- obs: long (nullable = false)
 |-- Weather_t: struct (nullable = false)
 |    |-- SkyCondition: string (nullable = true)
 |    |-- WindSpeed: string (nullable = true)
 |    |-- StationPressure: string (nullable = true)
 |    |-- Tmax: string (nullable = true)
 |    |-- Tmin: string (nullable = true)
 |    |-- AvgTemp: string (nullable = true)



In [62]:
// Vols au départ

val w_weatherD = Window
                .partitionBy("origin","fl_date","tail_num")
                .orderBy($"dep_time" .asc)

val w_rnD = Window
           .partitionBy("origin","fl_date","tail_num","dep_time")
           .orderBy($"dep_time" .asc)

val flightD_H12 = flightD_H_explode.withColumn("Weather_d", collect_list("Weather_t") over w_weatherD)
                                   .withColumn("rn", row_number over w_rnD )
                                   .where($"rn"===12)
                                   .select("cle","Weather_d")

print("Nb vols/Date après récupération 12 dernières heures infos météo : ")
println(flightD_H12.count())
      
flightD_H12.show(1, false)

Nb vols/Date après récupération 12 dernières heures infos météo : 461312
+------------------------------+------+----------+--------+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---+---+
|cle                           |origin|fl_date   |tail_num|dep_time|Weather_d                                                                                                                                                                                                                                                                                                                                      

w_weatherD: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@65d0633
w_rnD: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@473994e5
flightD_H12: org.apache.spark.sql.DataFrame = [cle: string, origin: string ... 6 more fields]


In [83]:
// Vols à l'arrivée

val w_weatherA = Window
                .partitionBy("dest","fl_date","tail_num")
                .orderBy($"arr_time" .asc)

val w_rnA = Window
           .partitionBy("dest","fl_date","tail_num","arr_time")
           .orderBy($"arr_time" .asc)

val flightA_H12 = flightA_H_explode.withColumn("Weather_a", collect_list("Weather_t") over w_weatherA)
                                   .withColumn("rn", row_number over w_rnA )
                                   .where($"rn"===12)
                                   .withColumnRenamed("cle","c_cle")
                                   .select("c_cle","dest", "fl_date", "tail_num","arr_time", "Weather_a")

print("Nb vols/Date après récupération 12 dernières heures infos météo : ")
println(flightA_H12.count())
      
flightA_H12.show(1,false)

Nb vols/Date après récupération 12 dernières heures infos météo : 463090
+------------------------------+----+----------+--------+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|c_cle                         |dest|fl_date   |tail_num|arr_time|Weather_a                                                                                                                                                                                                                                                                                                                                                                              

w_weatherA: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@702c856e
w_rnA: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@1021e0a2
flightA_H12: org.apache.spark.sql.DataFrame = [c_cle: string, dest: string ... 4 more fields]


## Etape 4/5 : regroupement des vols (départ / arrivée) via la clé => JT_201311

In [85]:
val flightDA_H12 = flightD_H12.join(flightA_H12, flightD_H12("cle")===flightA_H12("c_cle"), "inner")
                              .drop("c_cle")
                              .withColumnRenamed("cle","c_cle")

println(flightDA_H12.count())

val JT_201311 = FT_201311.join(flightDA_H12, 
                               FT_201311("cle") === flightDA_H12("c_cle"), "inner")
                              .drop("c_cle")

print("Table JT_201311. Nb vols/Date après récupération des données météo : ")
println(JT_201311.count())
JT_201311.show(1, false)

433248
Table JT_201311. Nb vols/Date après récupération des données météo : 433248
+------------------------------+----------+------+--------+----+--------+------------+---------+-------------+---------+---------------+----+--------+------------+---------+-------------+---------+-------------------+--------+--------+------+----------+--------+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

flightDA_H12: org.apache.spark.sql.DataFrame = [c_cle: string, origin: string ... 11 more fields]
JT_201311: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 30 more fields]


In [33]:
JT_201311.printSchema

root
 |-- origin: string (nullable = true)
 |-- fl_date: string (nullable = true)
 |-- tail_num: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- Weather_h: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- SkyCondition: string (nullable = true)
 |    |    |-- WindSpeed: string (nullable = true)
 |    |    |-- StationPressure: string (nullable = true)



## Etape 5/5 : transformation des variables Weather_d et Weather_a qui sont des array de struct en array
to be done...

In [ ]:
import spark.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row

// Sample data:
val df = Seq(
  ("id1", "t1", Array(("n1", 4L), ("n2", 5L))),
  ("id2", "t2", Array(("n3", 6L), ("n4", 7L)))
).toDF("ID", "Time", "Items")

// Create UDF converting array of (String, Long) structs to Map[String, Long]
val arrayToMap = udf[Map[String, Long], Seq[Row]] {
  array => array.map { case Row(key: String, value: Long) => (key, value) }.toMap
}

// apply UDF
val result = df.withColumn("Items", arrayToMap($"Items"))

result.show(false)

# 5. Label and target data creation

Evaluation of random forest algorithm is done on a combination of 4 parameters :
* Target dataset (D1, D2, D3, D4)
    * D1 contains only delayed flights due to extreme value weather or NAS, or a combination of them
    * D2 contains delayed flights due to extreme value weather + NAS delay if NAS delay > DT
    * D3 containes delayed flights due to extreme value weather or NAS, even if not exclusively
    * D4 contains all delayed flights
* Delay Threshold (DT)
    We consider 5 delay threshold (DT) in minutes : DT = 15,30,45,60,90)
* Number of hourly weather observations at origin airport (m) : m = 0,1,3,5,7,9,11
* Number of hourly weather observations at destination airport (n) : n = 0,1,3,45,7,9,11

In [13]:
val DT = Array(15,30,45,60,90)


// DT = 15
// Voir pour itérer automatiquement

val FT_target15 = data_vols2.withColumn("target_4", when(($"arr_delay" >= DT(0)) && ($"arr_delay" !== $"late_aircraft_delay"), 1).otherwise(0))
                      .withColumn("target_3", when(($"target_4" === 1) && (($"weather_delay" > 0) || ($"nas_delay" > 0)), 1).otherwise(0))
                      .withColumn("target_2", when(($"target_4" === 1) && (($"weather_delay" > 0) || ($"nas_delay"> DT(0))), 1).otherwise(0))
                      .withColumn("target_1", when(($"target_4" === 1) && ($"weather_delay" > 0) && ($"nas_delay" > 0) && 
                                                   ($"security_delay" === 0) && ($"carrier_delay" === 0), 1).otherwise(0))

val test = FT_target15.where($"target_4" === 1)
                      .withColumnRenamed("late_aircraft_delay","l_a_d")
                      //.where($"target_2"=== 1)
                      .select(
                             "arr_delay",
                             "weather_delay","nas_delay","security_delay","carrier_delay","l_a_d",
                             "target_1", "target_2", "target_3", "target_4"
                             ).show(50)

+---------+-------------+---------+--------------+-------------+-----+--------+--------+--------+--------+
|arr_delay|weather_delay|nas_delay|security_delay|carrier_delay|l_a_d|target_1|target_2|target_3|target_4|
+---------+-------------+---------+--------------+-------------+-----+--------+--------+--------+--------+
|    32.00|         0.00|    25.00|          0.00|         0.00| 7.00|       0|       1|       1|       1|
|   123.00|         0.00|   123.00|          0.00|         0.00| 0.00|       0|       1|       1|       1|
|    66.00|        10.00|    56.00|          0.00|         0.00| 0.00|       1|       1|       1|       1|
|    33.00|         0.00|    20.00|          0.00|        13.00| 0.00|       0|       1|       1|       1|
|    20.00|         0.00|    20.00|          0.00|         0.00| 0.00|       0|       1|       1|       1|
|    45.00|         0.00|    45.00|          0.00|         0.00| 0.00|       0|       1|       1|       1|
|    16.00|         0.00|    16.00|  

DT: Array[Int] = Array(15, 30, 45, 60, 90)
FT_target15: org.apache.spark.sql.DataFrame = [YEAR: string, QUARTER: string ... 68 more fields]
test: Unit = ()
